(gates)=

# Квантовые гейты

## Описание лекции

Из этой лекции мы узнаем:

- Какие есть основные однокубитные гейты
- Как записывать многокубитные состояния
- Как конструировать многокубитные операторы
- Какие есть многокубитные гейты
- Как работать с библиотекой `PennyLane`

## Введение

Квантовые гейты являются основными /строительными/ блоками для любых квантовых схем, в том числе и тех, которые применяются для машинного обучения. Можно сказать, что это такой алфавит квантовых вычислений, он просто необходим, чтобы сходу понимать, например, что изображено на подобных схемах:

```{figure} /_static/qcblock/gates/Layer-VQE.png
:name: lvqe
:width: 600px

[Схема Layered-VQE](https://arxiv.org/abs/2102.05566)
```

## Основные однокубитные гейты

В прошлый раз мы познакомились с {term}`операторами Паули<Операторы Паули>`, а также гейтом Адамара. Как для обычных квантовых алгоритмов, так и
для QML-алгоритмов нужны и другие гейты, потому что одни только эти гейты не позволяют перейти во все возможные квантовые состояния.
Теперь давайте посмотрим, какие еще однокубитные гейты часто применяются в квантовых вычислениях и квантовом машинном обучении.

### T-гейт

T-гейт очень популярен в универсальных квантовых вычислениях. Его матрица имеет вид

$$
\hat{T} = \begin{bmatrix}
1 & 0 \\
0 & \frac{1+i}{\sqrt{2}}
\end{bmatrix}
$$

Любой однокубитный гейт можно аппроксимировать последовательностью гейтов Адамара и T-гейтов. Чем точнее требуется аппроксимация, тем длиннее будет аппроксимирующая последовательность.

Помимо важной роли в математике квантовых вычислений, гейт Адамара и T-гейт интересны тем, что именно на них построено большинство предложений по реализации квантовых вычислений с топологической защитой или с коррекцией ошибок. На сегодняшний день эти схемы реально пока не очень работают: никаких топологически защищённых кубитов продемонстрировано не было, а коррекция ошибок не выходит за пределы двух логических кубитов.

### Гейты поворота вокруг оси

Поворотные гейты играют центральную роль в квантовом машинном обучении. Вспомним на секунду, как выглядят наши однокубитные состояния на сфере Блоха:

```{figure} /_static/qcblock/qubit/Blochsphere.svg
:name: blochsphere
:width: 400px

Сфера Блоха
```

Любой однокубитный гейт можно представить как вращение вектора состояния $\Psi$ на некоторый угол вокруг некоторой оси, проходящей через центр сферы Блоха.

Гейты $\hat{RX}(\phi), \hat{RY}(\phi), \hat{RZ}(\phi)$ осуществляют поворот на определенный угол $\phi$ вокруг соответствующей оси на сфере Блоха.

Давайте внимательно рассмотрим это на примере гейта $\hat{RY}$.

#### Гейт $\hat{RY}$

Сам гейт определяется следующим образом:

$$
\hat{RY}(\phi) = \begin{bmatrix}
\cos(\frac{\phi}{2}) & -\sin(\frac{\phi}{2}) \\
\sin(\frac{\phi}{2}) & \cos(\frac{\phi}{2})
\end{bmatrix}
$$

In [1]:
import numpy as np

def ry(state, phi):
    return np.array([
        [np.cos(phi / 2), -np.sin(phi / 2)],
        [np.sin(phi / 2),  np.cos(phi / 2)]
    ]) @ state

Запишем наше состояние $\ket{0}$:

In [2]:
basis = np.array([1 + 0j, 0 + 0j]).reshape((2, 1))

Внимательно посмотрев на сферу Блоха, можно заметить, что повернув состояние из $\ket{0}$ на $\pi$ и измерив значение $\hat{\sigma^z}$ мы получим 1, а повернув на $-\pi$ мы получим 0:

In [3]:
def expval(state, op):
    return state.conj().T @ op @ state

pauli_x = np.array([[0 + 0j, 1 + 0j], [1 + 0j, 0 + 0j]])

print(np.allclose(expval(ry(basis, np.pi / 2), pauli_x), 1.0))
print(np.allclose(expval(ry(basis, -np.pi / 2), pauli_x), -1.0))

True
True


Убедимся также, что вращение на угол, пропорциональный $2\pi$, не меняет результата измерения. Возьмем случайное состояние:

$$
\ket{\Psi} = \begin{bmatrix}
0.42 \\
\sqrt{1 - 0.42^2}
\end{bmatrix}
$$

In [4]:
random_state = np.array([0.42 + 0j, np.sqrt(1 - 0.42**2) + 0j]).reshape((2, 1))

Измерим его по осям $\mathbf{X}$ и $\mathbf{Z}$, затем повернем его на угол $2\pi$ и измерим снова:

In [5]:
pauli_z = np.array([[1 + 0j, 0 + 0j], [0 + 0j, 0j - 1]])

print("Z:\n\t" + str(expval(random_state, pauli_z)) + "\n")
print("X:\n\t" + str(expval(random_state, pauli_x)) + "\n")

print("Z after RY:\n\t" + str(expval(ry(random_state, 2 * np.pi), pauli_z)) + "\n")
print("X after RY:\n\t" + str(expval(ry(random_state, 2 * np.pi), pauli_x)) + "\n")

Z:
	[[-0.6472+0.j]]

X:
	[[0.76232025+0.j]]

Z after RY:
	[[-0.6472+0.j]]

X after RY:
	[[0.76232025+0.j]]



#### Другие гейты вращений

Аналогичным образом определяются гейты $\hat{RX}$ и $\hat{RZ}$:

$$
\hat{RX}(\phi) = \begin{bmatrix}
\cos(\frac{\phi}{2}) & -i\sin(\frac{\phi}{2}) \\
-i\sin(\frac{\phi}{2}) & \cos(\frac{\phi}{2})
\end{bmatrix} \qquad \hat{RZ}(\phi) = \begin{bmatrix}
e^{-\frac{i\phi}{2}} & 0 \\
0 & e^{\frac{i\phi}{2}}
\end{bmatrix}
$$

#### Общая форма записи однокубитных гейтов

В общем случае однокубитные гейты могут быть также записаны следующим образом:

$$
\large \hat{R}^\vec{n}(\alpha) = e^{-\frac{i\alpha\hat{\vec{\sigma}}\vec{n}}{2}},
$$

где $\alpha$ -- это угол поворота, $\vec{n}$ -- единичный вектор в направлении оси поворота, а $\hat{\vec{\sigma}} = \{\hat{\sigma}^x, \hat{\sigma}^y, \hat{\sigma}^z\}$ -- это вектор, составленный из операторов Паули. Если использовать покоординатную запись и $\vec{n} = \{n_x, n_y, n_z\}$ задаёт ось вращения, то

$$
\large \hat{R}^\vec{n}(\alpha) = e^{-i\frac{\alpha}{2}\left(\hat{\sigma}^xn_x+\hat{\sigma}^yn_y+\hat{\sigma}^zn_z\right)}.
$$

Забегая вперед, можно сказать, что именно гейты вращений -- это основа квантовых вариационных схем, главного инструмента этого курса.

### Phase-shift гейт

Другой важный гейт -- это так называемый phase-shift гейт, или $\hat{U}_1$ гейт. Его матричная форма имеет следующий вид:

$$
\hat{U}_1(\phi) = \begin{bmatrix}
1 & 0 \\
0 & e^{i\phi}
\end{bmatrix}
$$

In [6]:
def u1(state, phi):
    return np.array([[1, 0], [0, np.exp(1j * phi)]]) @ state

Легко видеть, что с точностью до глобального фазового множителя, который ни на что не влияет, Phase-shift-гейт -- это тот же $\hat{RZ}(\phi)$.
Забегая вперед, можно сказать, что этот гейт играет важную роль в квантовых ядерных методах.

### Гейты $\hat{U}_2$ и $\hat{U}_3$

Более редкие в QML гейты, которые однако все равно встречаются в статьях.

$$
\hat{U}_2(\phi, \lambda) = \frac{1}{\sqrt{2}}\begin{bmatrix}
1 & -e^{i\lambda} \\
e^{i\phi} & e^{i(\phi + \lambda)}
\end{bmatrix} = \hat{U}_1(\phi + \lambda)\hat{RZ}(-\lambda)\hat{RY}(\frac{\pi}{2})\hat{RZ}(\lambda)
$$

Давайте убедимся в справедливости этого выражения:

In [7]:
def rz(state, phi):
    return np.array([[np.exp(-1j * phi / 2), 0], [0, np.exp(1j * phi / 2)]]) @ state


def u2_direct(phi, l):
    return (
        1
        / np.sqrt(2)
        * np.array([[1, -np.exp(1j * l)], [np.exp(1j * phi), np.exp(1j * (phi + l))]])
    )


def u2_inferenced(phi, l):
    return (
        u1(np.eye(2), phi + l)
        @ rz(np.eye(2), -l)
        @ ry(np.eye(2), np.pi / 2)
        @ rz(np.eye(2), l)
    )

print(np.allclose(u2_direct(np.pi / 6, np.pi / 3), u2_inferenced(np.pi / 6, np.pi / 3)))

True


Схожим образом определяется $\hat{U}_3(\theta, \phi, \lambda)$:

$$
\hat{U}_3(\theta, \phi, \lambda) = \begin{bmatrix}
\cos(\frac{\theta}{2}) & -e^{1j\lambda}\sin(\frac{\theta}{2}) \\
e^{1j\phi}\sin(\frac{\theta}{2}) & e^{1j(\phi + \lambda)}\cos(\frac{\theta}{2})
\end{bmatrix} = \hat{U}_1(\phi + \lambda)\hat{RZ}(-\lambda)\hat{RY}(\theta)\hat{RZ}(\lambda)
$$

Читатель может сам легко убедиться, что эти формы записи эквивалентны. Для этого надо написать примерно такой же код, какой мы писали раньше для $\hat{U}_2$.

### Еще пара слов об однокубитных гейтах

На этом мы завершаем обзор основных однокубитных гейтов. Маленькое замечание -- гейты, связанные со сдвигом фазы, никак не меняют состояние кубита, если оно сейчас $\ket{0}$. Учитывая, что мы всегда предполагаем, что наше начальное состояние кубитов -- это именно $\ket{0}$, то перед тем как применять, например, гейт $\hat{U}_1$, рекомендуется применить гейт Адамара:

In [8]:
print(np.allclose(u1(basis, np.pi / 6), basis))

h = 1 / np.sqrt(2) * np.array([[1 + 0j, 1 + 0j], [1 + 0j, 0j - 1]])
print(np.allclose(u1(h @ basis, np.pi / 6), h @ basis))

True
False


#### Единичный гейт

Самое последнее об однокубитных гейтах -- это единичный гейт $\hat{I}$:

$$
\hat{I} = \begin{bmatrix}
1 & 0 \\
0 & 1
\end{bmatrix}
$$

In [9]:
i = np.eye(2, dtype=np.complex128)
print(i)

[[1.+0.j 0.+0.j]
 [0.+0.j 1.+0.j]]


Этот гейт не делает с кубитом ровным счетом ничего. Но он все равно нам понадобится, когда мы будем конструировать многокубитные операторы!

## Многокубитные состояния и гейты

Очевидно, что с одним кубитом мы ничего интересного (кроме генератора истинно-случайных чисел) не сделаем. Давайте для начала разберемся, как выглядят состояния для многокубитных систем.

### Многокубитные состояния

В классическом компьютере 1 бит имеет два значения -- 0 и 1; 2 бита имеют 4 -- 00, 01, 10, 11; 3 бита 8 значений и т.д. По аналогии, состояние двух кубитов -- это вектор в пространстве $\mathbf{C}^4$; состояние трех кубитов, соответственно, в пространстве $\mathbf{C}^8$ и т.д., то есть состояние $N$ кубитов описывается вектором размерности $2^N$ в комплексном пространстве. Вероятности каждой из возможных битовых строк ($0000...00$, $0000...01$, $0000...10$, и т.д.) получаются по методу Шрёдингера, который мы обсуждали в конце прошлой лекции:

$$
\mathbf{P}(\vec{s}) = | \bra{\Psi}\ket{\vec{s}} |^2
$$

Напомню, что по сути мы должны отсортировать наши битовые строки в лексикографическом порядке, и вероятность _i_-й битовой строки будет равна квадрату _i_-го элемента вектора $\ket{\Psi}$.

Формально, многокубитные состояния описываются с помощью математического концепта так называемого тензорного произведения, или по-другому, произведения Кронекера, обозначаемого значком $\otimes$. Так, если $\ket{\Psi}_A \in \mathrm{H}_A$ и $\ket{\Psi}_B \in \mathrm{H}_B$, то $\ket{\Psi}_{AB} = \ket{\Psi}_A \otimes \ket{\Psi}_B \in \mathrm{H}_{AB} = \mathrm{H}_{A} \otimes \mathrm{H}_{B}$. Как элементы вектора $\ket{\Psi}_{AB}$ выражаются через элементы векторов $\ket{\Psi}_{A}$ и $\ket{\Psi}_{B}$, можно прочитать на Википедии в статье ["Произведение Кронекера"](https://ru.wikipedia.org/wiki/Произведение_Кронекера/).

### Многокубитные операторы

Как мы уже обсуждали, квантовые операторы должны переводить наше состояние в новое состояние в том же пространстве и сохранять нормировку, а еще должны быть обратимыми. И, значит, оператор для состояния из $N$ кубитов -- это унитарная комплексная матрица размерности $2^N \times 2^N$.

### Конструирование многокубитных операторов

Прежде чем мы начнем обсуждать двухкубитные операторы, давайте рассмотрим другую ситуацию. Представим, что у нас есть состояние из 2-х кубитов и мы хотим подействовать на первый кубит, например, оператором Адамара. Как же тогда нам написать такой двухкубитный оператор? Ок, мы действуем на 1-й кубит оператором, а что происходит со вторым кубитом? А ничего не происходит, и это эквивалентно тому, что мы действуем на 2-й кубит единичным оператором. А финальный оператор $2^2 \times 2^2$ записывается через произведение Кронекера:

$$
\hat{H} \otimes \hat{I} = \frac{1}{\sqrt{2}}\begin{bmatrix}
1 & 1 \\
1 & -1
\end{bmatrix} \otimes \begin{bmatrix}
1 & 0 \\
0 & 1
\end{bmatrix} = \frac{1}{\sqrt{2}} \begin{bmatrix}
\hat{I} & \hat{I} \\
\hat{I} & -\hat{I}
\end{bmatrix} = \frac{1}{\sqrt{2}} \begin{bmatrix}
1 & 0 & 1 & 0 \\
0 & 1 & 0 & 1 \\
1 & 0 & -1 & 0 \\
0 & 1 & 0 & -1
\end{bmatrix}
$$

Учитывая, что многокубитные состояния конструируются аналогичным образом через произведение Кронекера, мы можем явно убедиться в верности нашего вывода:

In [10]:
print(np.allclose(np.kron(h @ basis, basis), np.kron(h, i) @ np.kron(basis, basis)))

True


### Наблюдаемые для многокубитных гейтов

Аналогичным образом можно сконструировать и наблюдаемые. Например, если мы хотим измерять одновременно два спина по оси $\mathbf{Z}$, то наблюдаемая будет выглядеть так:

$$
\mathbf{ZZ} = \hat{\sigma^z} \otimes \hat{\sigma^z} = \begin{bmatrix}
1 & 0 & 0 & 0 \\
0 & -1 & 0 & 0 \\
0 & 0 & -1 & 0 \\
0 & 0 & 0 & 1
\end{bmatrix}
$$

In [11]:
print(np.kron(basis, basis).conj().T @ np.kron(pauli_z, pauli_z) @ np.kron(basis, basis))

[[1.+0.j]]


## Основные двухкубитные гейты

Основные многокубитные гейты, которые предоставляют современные квантовые компьютеры, -- это двухкубитные гейты.

### CNOT (CX)

Квантовый гейт контролируемого инвертирования -- это гейт, который действует на два кубита -- /рабочий/ и /контрольный/. В зависимости от того, имеет ли контрольный кубит значение 1 или 0, этот гейт инвертирует или не инвертирует рабочий кубит.

```{figure} /_static/qcblock/gates/CNOT_gate.svg
:name: cnot
:width: 400px

Гейт CNOT
```

Иногда этот гейт также называют гейтом CX. В матричном виде этот оператор можно записать так:

$$
\hat{CNOT} = \begin{bmatrix}
1 & 0 & 0 & 0\\
0 & 1 & 0 & 0\\
0 & 0 & 0 & 1\\
0 & 0 & 1 ,& 0
\end{bmatrix}
$$

In [12]:
cnot = (1 + 0j) * np.array(
    [
        [1, 0, 0, 0],
        [0, 1, 0, 0],
        [0, 0, 0, 1],
        [0, 0, 1, 0],
    ]
)

print(np.allclose(cnot @ np.kron(basis, basis), np.kron(basis, basis)))
print(np.allclose(
    cnot @ np.kron(pauli_x @ basis, basis), np.kron(pauli_x @ basis, pauli_x @ basis)
))

True
True


Заметьте, тут мы воспользовались тем, что $\hat{\sigma^x}$ работает так же, как инвертор кубитов -- он превращает $\ket{0}$ в $\ket{1}$ и наоборот!

### Гейты CY и CZ

Схожие по принципу гейты -- это гейты $\hat{CY}$ и $\hat{CZ}$. В зависимости от значения /управляющего/ кубита к /рабочему/ кубиту применяют соответствующий оператор Паули:

$$
\hat{CY} = \begin{bmatrix}
1 & 0 & 0 & 0\\
0 & 1 & 0 & 0\\
0 & 0 & 0 & -i\\
0 & 0 & i & 0
\end{bmatrix} \qquad \hat{CZ} = \begin{bmatrix}
1 & 0 & 0 & 0\\
0 & 1 & 0 & 0\\
0 & 0 & 1 & 0\\
0 & 0 & 0 & -1
\end{bmatrix}
$$

### Гейт iSWAP

Гейты $\hat{CX}$, $\hat{CY}$ и $\hat{CZ}$ эквивалентны с точностью до однокубитных гейтов. Это означает, что любой из них можно получить, добавив необходимые однокубитные гейты до и после другого гейта. Так, например,

$$
\hat{CZ} = \left(\hat{I}\times\hat{H}\right)\hat{CX}\left(\hat{I}\times\hat{H}\right).
$$

Этим свойством обладают отнюдь не все двухкубитные гейты! Например, таковым является гейт iSWAP:

$$
\mathrm{iSWAP} = \begin{bmatrix}
1 & 0 & 0 & 0\\
0 & 0 & -i & 0\\
0 & -i & 0 & 0\\
0 & 0 & 0 & 1
\end{bmatrix}
$$

### Гейт fSim

Для разных архитектур квантовых процессоров "естественный" гейт может выглядеть по-разному. Так, например, в квантовом процессоре Google Sycamore естественным является так называемый fermionic simulation gate, или fSim. Это двухпараметрическое семейство гейтов вида

$$
\mathrm{fSim}(\theta, \phi) = \begin{bmatrix}
1 & 0 & 0 & 0\\
0 & \cos\theta & -i\sin\theta & 0\\
0 & -i\sin\theta & \cos\theta & 0\\
0 & 0 & 0 & e^{-i\phi}
\end{bmatrix}.
$$

Впрочем, и fSim-гейт не является эквивалентным всему множеству двухкубитных гейтов. В общем случае, чем больше кубитов,
тем сложнее будет выглядеть декомпозиция произвольного гейта на физически реализуемые в железе.

## Первое знакомство с PennyLane

На сегодня существует уже достаточно много фреймворков для программирования квантовых компьютеров. Для целей этого курса мы будем использовать [PennyLane](https://pennylane.ai/). Он предоставляет высокоуровневый `Python API` и создан специально для решения задач квантового машинного обучения.

In [13]:
import pennylane as qml

### Device

Для объявления квантового устройства используется класс `Device`. `PennyLane` поддерживает работу с большинством существующих сегодня квантовых компьютеров, но для целей курса мы будем запускать все наши программы лишь на самом простом симуляторе идеального квантового компьютера:

In [14]:
device = qml.device("default.qubit", 2)

Первый аргумент тут -- указание устройства, а второй -- число кубитов.

### QNode

Основной _строительный блок_ в `PennyLane` -- это `qnode`. Это функция, которая отмечена специальным декоратором и включает в себя несколько операций с кубитами. Результатом такой функции всегда является измерение. Напишем функцию, которая поворачивает первый кубит на $45^o$, после чего измеряет оба кубита по оси $\mathbf{Z}$.

#### Сначала на NumPy

In [15]:
state = np.kron(basis, basis)
op = np.kron(ry(np.eye(2), np.deg2rad(45)), np.eye(2, dtype=np.complex128))
measure = np.kron(pauli_z, pauli_z)

print((op @ state).conj().T @ measure @ (op @ state))

[[0.70710678+0.j]]


#### Теперь через QNode

In [16]:
@qml.qnode(device)
def test(angle):
    qml.RY(angle, wires=0)
    return qml.expval(qml.PauliZ(0) @ qml.PauliZ(1))


print(test(np.deg2rad(45)))

0.7071067811865475


## Заключение

Это последняя вводная лекция, где мы сами писали операторы и операции на чистом `NumPy`. Это могло казаться скучным, но зато должно помочь лучше понять ту математику, которая лежит под капотом у квантовых вычислений. Дальше мы будем пользоваться только `PennyLane` и будет отдельная лекция, которая рассказывает, как работать с этим фреймворком.

- Мы знаем что такое кубит
- Мы понимаем линейную алгебру, которая описывает квантовые вычисления
- Мы понимаем, как можно сконструировать нужный нам оператор и как применить его
- Мы знаем, что такое измерение и наблюдаемые

Теперь мы готовы к тому, чтобы знакомиться с квантовыми вариационными схемами и переходить непосредственно к построению моделей квантового машинного обучения!

## Задачи

- Как связаны ось и угол вращения на сфере Блоха с собственными значениями и собственными векторами матрицы однокубитного гейта? Для этого найдите собственные векторы и собственные значения гейта $R^\vec{n}\left(\alpha\right)$.
- Вокруг какой оси и на какой угол вращает состояние гейт Адамара?
- Гейт SWAP меняет кубиты местами. Его унитарная матрица имеет вид

$$
\mathrm{SWAP} =
\begin{bmatrix}
1 & 0 & 0 & 0 \\
0 & 0 & 1 & 0 \\
0 & 1 & 0 & 0 \\
0 & 0 & 0 & 1
\end{bmatrix}.
$$

Попробуйте составить последовательность гейтов, реализующую $\mathrm{SWAP}$, из гейтов $\mathrm{iSWAP}$, $\hat{CZ}$ и $\hat{RZ}(\phi)$.